In [7]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [10]:
model_version=4
new_stage = "Staging"
run_id = "e7dc46a4a7d24cecab0ea999910280f5"
model_uri=f"runs:/{run_id}/model"
model_name = "nyc-taxi-regressor"

In [ ]:
client.search_experiments()

[<Experiment: artifact_location='/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1748522465340, experiment_id='1', last_update_time=1748522465340, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1748522052060, experiment_id='0', last_update_time=1748522052060, lifecycle_stage='active', name='Default', tags={}>]

In [9]:
client.create_experiment(name="my-new-experiment")

MlflowException: Experiment(name=my-new-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-new-experiment', None, 'active', 1748546071836, 1748546071836)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [ ]:
client.search_experiments()

[<Experiment: artifact_location='/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1748543686265, experiment_id='2', last_update_time=1748543686265, lifecycle_stage='active', name='my-new-experiment', tags={}>,
 <Experiment: artifact_location='/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1748522465340, experiment_id='1', last_update_time=1748522465340, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1748522052060, experiment_id='0', last_update_time=1748522052060, lifecycle_stage='active', name='Default', tags={}>]

In [ ]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="", #  "metrics.rmse < 6.31",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [ ]:
for run in runs:
    print(f"run_id: {run.info.run_id}, rmse = {run.data.metrics['rmse']:.4f}")

run_id: e7dc46a4a7d24cecab0ea999910280f5, rmse = 6.3059
run_id: a1f723941ca0459ba944634e0d917ca9, rmse = 6.3059
run_id: 754fd769910a44f590153eb1bceabead, rmse = 6.3096
run_id: 251f1f20f59246cf87c314961a0f2f41, rmse = 6.3282
run_id: c8bc758fb5af4128a5a85437d20a5d3b, rmse = 6.3795


In [ ]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:

mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '5' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1748545765617, current_stage='None', description=None, last_updated_timestamp=1748545765617, name='nyc-taxi-regressor', run_id='e7dc46a4a7d24cecab0ea999910280f5', run_link=None, source='/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/mlruns/1/e7dc46a4a7d24cecab0ea999910280f5/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [34]:
latest_versions = client.get_latest_versions(name=model_name)
# latest_versions = client.get_model_version_by_alias(name=model_name, alias="archived")

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")
# latest_versions

version: 4, stage: Staging
version: 2, stage: Production
version: 3, stage: None


/var/folders/zq/ltw42k9s1q9dcwdks71_z86c0000gn/T/ipykernel_21699/7117234.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [ ]:

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/var/folders/zq/ltw42k9s1q9dcwdks71_z86c0000gn/T/ipykernel_21699/2336212361.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1748544410573, current_stage='Staging', description=None, last_updated_timestamp=1748545337380, name='nyc-taxi-regressor', run_id='a1f723941ca0459ba944634e0d917ca9', run_link=None, source='/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/mlruns/1/a1f723941ca0459ba944634e0d917ca9/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [37]:
from datetime import datetime
date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1748544410573, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2025-05-29', last_updated_timestamp=1748545338688, name='nyc-taxi-regressor', run_id='a1f723941ca0459ba944634e0d917ca9', run_link=None, source='/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/mlruns/1/a1f723941ca0459ba944634e0d917ca9/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [32]:
import pandas as pd
from sklearn.metrics import root_mean_squared_error

def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
    
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

def test_model_mlflow(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [19]:
df = read_dataframe("./data/green_tripdata_2021-03.parquet")

In [20]:
client.download_artifacts(run_id=run_id, path='processor', dst_path='.')

'/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/processor'

In [21]:
import pickle 

with open("processor/processor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [22]:
X_test = preprocess(df, dv)


In [23]:
target = "duration"
y_test = df[target].values

In [28]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 316 ms, sys: 10.1 ms, total: 326 ms
Wall time: 327 ms


{'rmse': 6.648246909512727}

In [35]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 19.5 s, sys: 134 ms, total: 19.7 s
Wall time: 2.28 s


{'rmse': 6.257863027498054}

In [36]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production", 
    archive_existing_versions=True
)

/var/folders/zq/ltw42k9s1q9dcwdks71_z86c0000gn/T/ipykernel_22139/1517243906.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1748544410573, current_stage='Production', description='The model version 4 was transitioned to Staging on 2025-05-29', last_updated_timestamp=1748547261500, name='nyc-taxi-regressor', run_id='a1f723941ca0459ba944634e0d917ca9', run_link=None, source='/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/mlruns/1/a1f723941ca0459ba944634e0d917ca9/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>